In [ ]:
# ---EXERCISE 2, A:

In [1]:
inputPath = "/user/s292129/data/POIs.txt"
outputPath = "res_out_ExExemple1/"

In [5]:
inputRDD = sc.textFile(inputPath).cache()
filteredRDD = inputRDD.filter(lambda line: line.split(',')[4] == "Italy")
city_subcatRDD = filteredRDD.map(lambda line: (line.split(',')[3],\
                                               line.split(',')[6]))


In [6]:
groupedByCityRDD = city_subcatRDD.groupByKey()
#groupedByCityRDD.mapValues(lambda l: list(l)). ()

[('Milan', ['taxi', 'busstop']),
 ('Carmagnola', ['taxi']),
 ('Turin',
  ['museum',
   'museum',
   'artgallery',
   'shoes',
   'shoes',
   'shoes',
   'taxi',
   'busstop',
   'taxi']),
 ('Rome', ['museum', 'artgallery', 'artgallery']),
 ('Naples', ['shoes'])]

In [7]:
lastFiltered = groupedByCityRDD.filter(lambda pair: "taxi" in pair[1] and\
                                       "busstop" not in pair[1])

In [9]:
lastFiltered.keys().saveAsTextFile(outputPath)

In [26]:
#----EXERCISE 2 , B:

def mapCountryCounters(pair):
    if "museum" in pair[1]:
        return (1,1)
    else:
        return (0,1)
    
    
countersRDD = groupedByCityRDD.map(mapCountryCounters)

In [29]:
pair = countersRDD.reduce(lambda pair1, pair2: (pair1[0]+pair2[0], pair1[1]+pair2[1]))
averageMuseumsPerCity = pair[0]/pair[1]

In [35]:
def mapValuePerMuseums(value):
    if value=="museum":
        return 1
    else:
        return 0
    
    
cityMuseum = city_subcatRDD.mapValues(mapValuePerMuseums)

In [36]:
cityNumMuseum = cityMuseum.reduceByKey(lambda v1,v2: v1+v2)

In [39]:
cityNumMuseum.filter(lambda pair: pair[1] > averageMuseumsPerCity).keys().saveAsTextFile(outputPath)

In [40]:
#-----EXERCISE 2, B / ALTERNATIVE:

# DEFINE DF and rename columns
def museums(subcategory):
    if subcategory=="museum":
        return 1
    else:
        return 0
    
spark.udf.register("numMuseums",museums)

<function __main__.museums(subcategory)>

In [ ]:
cityNumMuseumsDF = spark.sql("SELECT city, SUM(numMuseums(subcategory))\
            FROM inputData WHERE subcategory='museum'\
            GROUP BY city")

#query the average numMuseum in italy and then filter the cities....